* Data pipeline for the image ETL

In [1383]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.VERSION
AUTOTUNE = tf.contrib.data.AUTOTUNE

In [1384]:
import pathlib
data_root = pathlib.Path('./data/train/')
print(data_root)
for item in data_root.iterdir():
  print(item)

data/train
data/train/barramundi
data/train/snapper
data/train/bream


In [1385]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count
all_image_paths[:10]

['data/train/bream/bream3.jpg',
 'data/train/bream/bream4.jpeg',
 'data/train/snapper/snapper4.jpeg',
 'data/train/snapper/Snapper-Medium-Whole.jpg',
 'data/train/snapper/Snapper6.jpg',
 'data/train/snapper/snapper3-660x400.jpg',
 'data/train/barramundi/Barramundi5.jpg',
 'data/train/barramundi/Barramundi-Medium.jpg',
 'data/train/snapper/snapper1.jpeg',
 'data/train/snapper/bream1.png']

In [1386]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['barramundi', 'bream', 'snapper']

In [1387]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

{'barramundi': 0, 'bream': 1, 'snapper': 2}

In [1388]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:6])

First 10 labels indices:  [1, 1, 2, 2, 2, 2]


In [1389]:
# import IPython.display as display
# for n in range(2):
#   image_path = random.choice(all_image_paths)
#   display.display(display.Image(image_path))
# #   print(caption_image(image_path))
#   print()

The preprocessing include a resize action. Currently we are using squash method - ignore the ration and turn it to square. Please check [this article](https://forums.fast.ai/t/impact-of-image-resizing-on-model-training-time-and-performance/1980) to see the comparison amoung the different methods of resizing

In [1390]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize_images(image, [128, 128])
  image /= 255.0  # normalize to [0,1] range

  return image

In [1391]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

Take a look what happen after the image processing.

In [1392]:
# import matplotlib.pyplot as plt

# image_path = all_image_paths[1]
# label = all_image_labels[1]

# load_and_preprocess_image(image_path)

# plt.imshow(load_and_preprocess_image(image_path))
# plt.grid(False)
# plt.title(label_names[label].title())
# plt.show()

Start preparing the dataset

In [1393]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [1394]:
# image_ds = all_image_paths.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
# image_ds
# label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
# label_ds

In [1395]:
# for label in label_ds.take(10):
#   print(label_names[label.numpy()])

In [1396]:
# image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [1397]:
train_images = []
train_labels = []
for image_path in all_image_paths:
    train_images.append(load_and_preprocess_image(image_path))
for label_item in tf.cast(all_image_labels, tf.int64):
    train_labels.append(label_item)
ds=tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(2).repeat()
    

Now Make the model ready

In [1398]:
# BATCH_SIZE = 32

# # Setting a shuffle buffer size as large as the dataset ensures that the data is
# # completely shuffled.
# ds = image_label_ds.shuffle(buffer_size=image_count)
# ds = ds.repeat()
# ds = ds.batch(BATCH_SIZE)
# # `prefetch` lets the dataset fetch batches, in the background while the model is training.
# ds = ds.prefetch(buffer_size=AUTOTUNE)
# ds

In [1399]:
from tensorflow import keras
# import keras.layers.Conv2D as Conv2D
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(32, 32, 3)),
#     keras.layers.Dense(128, activation=tf.nn.relu),
#     keras.layers.Dense(3, activation=tf.nn.softmax)
# ])

IMG_SIZE = 32
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(3, activation = 'softmax'))

In [1400]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [1401]:
model.fit(ds, epochs=50, steps_per_epoch=1)

Epoch 1/50
1/1 [==============================] - 0s 149ms/step - loss: 0.5644 - acc: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 91ms/step - loss: 1.3718 - acc: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 77ms/step - loss: 1.2827 - acc: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 82ms/step - loss: 1.2796 - acc: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 83ms/step - loss: 1.1272 - acc: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 84ms/step - loss: 0.8807 - acc: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 92ms/step - loss: 1.1675 - acc: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 82ms/step - loss: 1.7566 - acc: 0.5000
Epoch 9/50
1/1 [==============================] - 0s 81ms/step - loss: 1.4007 - acc: 0.0000e+00
Epoch 10/50
1/1 [==============================] - 0s 81ms/step - loss: 4.3247 - acc: 0.0000e+00
Epoch 11/50
1/1 [==============================] - 0s 81ms/step - 

In [1402]:
test_loss, test_acc = model.evaluate(ds, steps=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

1/1 [==============================] - 0s 34ms/step
Test loss: 0.9597550630569458
Test accuracy: 1.0
